In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [240]:
from ipywidgets import widgets

In [2]:
%matplotlib inline

In [119]:
table1 = pd.read_csv('USD000000TOD.trades_20180514_092903.csv', sep=';')
table1.head()

,quantity,price,isBuy,ticks,origTimeHHMMSS,origTimeMicroSecs,fastId,fixId,tag779,tag52,...,srcPx,futStatus,priceInfo,dstPriceAdd,timePacket,timeCreate,timeStartPrepare,timeEndPrepare,delta,Unnamed: 21
0,10,6189000,1,NaN,70000,766582,168639238,-1,0,180514070000767140,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
1,10,6189000,1,NaN,70000,770012,168639239,-1,0,180514070000770445,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
2,9,6189000,1,NaN,70000,792681,168639242,-1,0,180514070000793141,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
3,51,6189000,1,NaN,70001,157832,168639246,-1,0,180514070001158366,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
4,1,6186750,0,NaN,70001,310769,168639247,-1,0,180514070001311246,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN


In [5]:
table2 = pd.read_csv('USD000UTSTOM.trades_20180514_092903.csv', sep=';')
table2.head()

,quantity,price,isBuy,ticks,origTimeHHMMSS,origTimeMicroSecs,fastId,fixId,tag779,tag52,...,srcPx,futStatus,priceInfo,dstPriceAdd,timePacket,timeCreate,timeStartPrepare,timeEndPrepare,delta,Unnamed: 21
0,1,6192500,1,NaN,70000,600900,168639234,-1,0,180514070000601372,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
1,10,6185750,0,NaN,70000,623106,168639235,-1,0,180514070000623608,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
2,1,6188500,0,NaN,70000,681441,168639236,-1,0,180514070000681988,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
3,4,6188500,0,NaN,70000,716771,168639237,-1,0,180514070000717315,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN
4,10,6193000,0,NaN,70000,772476,168639240,-1,0,180514070000772857,...,0,-1,94484348479664,140219878265944,0.0,0.0,0.0,0.0,0,NaN


In [293]:
def my_func(table, window_size):
    
    w_size = window_size*1000000*60 # microsec -> 1 min
    
    table.columns = map(lambda x: x.strip(), table.columns)

    table['fulltime'] = table.apply(lambda x: x['origTimeHHMMSS']*1000000 + x['origTimeMicroSecs'], axis=1).astype('int64')

    min_time = table.fulltime.min()

    table['w_quantity'] = table.apply(lambda x: x['quantity'] if x['isBuy'] == 1 else -x['quantity'], axis=1)

    grouped_by_time = table.groupby(['fulltime']).agg({'w_quantity': 'sum'}).reset_index()

    grouped_by_time['window_number'] = grouped_by_time.fulltime.apply(lambda x: (x - min_time) // w_size)


    grouped_by_window = grouped_by_time.groupby(by=['window_number']).agg({'w_quantity' : 'sum', 'window_number' : 'count'})



    grouped_by_window.columns = ['sum_quantity', 'w_size']
    grouped_by_window['omega'] = grouped_by_window.apply(lambda x: x.sum_quantity/x.w_size, axis=1)

    common_df = grouped_by_time.join(grouped_by_window, on='window_number')

    common_df['sigma'] = common_df.apply(lambda x: (x['w_quantity'] - x['omega'])/x['w_size'], axis=1)

    return common_df[['fulltime', 'window_number', 'omega', 'sigma']]

In [294]:
my_func(table, window_size)

,fulltime,window_number,omega,sigma
0,70000600900,0,4.652893,-0.030189
1,70000623106,0,4.652893,-0.121098
2,70000681441,0,4.652893,-0.046718
3,70000716771,0,4.652893,-0.071512
4,70000772476,0,4.652893,-0.121098
5,70000779607,0,4.652893,-0.021925
6,70000794958,0,4.652893,-0.112834
7,70000816862,0,4.652893,-0.054983
8,70000928041,0,4.652893,-0.063247
9,70001787697,0,4.652893,0.192951


In [277]:
window_size=1
table = pd.read_csv('USD000UTSTOM.trades_20180514_092903.csv', sep=';')

In [279]:
w_size = window_size*1000000*60 # microsec -> 1 min
    
table.columns = map(lambda x: x.strip(), table.columns)

table['fulltime'] = table.apply(lambda x: x['origTimeHHMMSS']*1000000 + x['origTimeMicroSecs'], axis=1).astype('int64')

min_time = table.fulltime.min()

table['w_quantity'] = table.apply(lambda x: x['quantity'] if x['isBuy'] == 1 else -x['quantity'], axis=1)

grouped_by_time = table.groupby(['fulltime']).agg({'w_quantity': 'sum'}).reset_index()

grouped_by_time['window_number'] = grouped_by_time.fulltime.apply(lambda x: (x - min_time) // w_size)


grouped_by_window = grouped_by_time.groupby(by=['window_number']).agg({'w_quantity' : 'sum', 'window_number' : 'count'})


grouped_by_window.columns = ['sum_quantity', 'w_size']
grouped_by_window['omega'] = grouped_by_window.apply(lambda x: x.sum_quantity/x.w_size, axis=1)

common_df = grouped_by_time.join(grouped_by_window, on='window_number')

common_df['sigma'] = common_df.apply(lambda x: (x['w_quantity'] - x['omega'])/x['w_size'], axis=1)

final_df = common_df[['fulltime', 'window_number', 'omega', 'sigma']]

In [236]:
hz_che = final_df.groupby('window_number')['sigma'].sum()

In [239]:
hz_che[hz_che > 5]

Series([], Name: sigma, dtype: float64)

In [269]:
checkbox = widgets.interact(f, x=10)
checkbox;

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-interact',))

In [254]:
def f(x):
    print(x)

In [272]:
checkbox

<function __main__.f>